## 얼굴 검출(이미지)

In [1]:
import numpy as np
import cv2

In [26]:
# 분류기 생성
face_cascade = cv2.CascadeClassifier('./data/haarcascade_frontalface_default.xml')

img = cv2.imread('./img/children.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray)

for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    roi = gray[y:y+h, x:x+w]

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 얼굴 검출(카메라)

In [21]:
import cv2

In [22]:
# 분류기 생성
face_cascade = cv2.CascadeClassifier('./data/haarcascade_frontalface_default.xml')

In [28]:
cap = cv2.VideoCapture(-1)
while cap.isOpened():
    ret, img = cap.read()
    if ret:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.3, minNeighbors = 5, minSize = (80, 80))
        
        for(x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            roi = gray[y:y+h, x:x+w]
        
        cv2.imshow('face detect', img)
    else:
        break
    if cv2.waitKey(5) == 27:
        break
cv2.destroyAllWindows()

## LBP 얼굴 인식을 위한 샘플 수집

In [13]:
import cv2
import numpy as np
import os

In [30]:
base_dir = './faces/'
target_cnt = 400
cnt = 0

# 분류기 생성
face_classifier = cv2.CascadeClassifier('./data/haarcascade_frontalface_default.xml')

name = input("Insert User Name(Only Alphabet):")
id = input("Insert User Id(Non-Duplicate number):")
dir = os.path.join(base_dir, name+'_' + id)
if not os.path.exists(dir):
    os.mkdir(dir)

cap = cv2.VideoCapture(-1)
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        img = frame.copy()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 1:
            (x, y, w, h) = faces[0]
            
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 1)
            face = gray[y:y+h, x:x+w]
            face = cv2.resize(face, (200, 200))
            file_name_path = os.path.join(dir, str(cnt) + '.jpg')
            cv2.imwrite(file_name_path, face)
            cv2.putText(frame, str(cnt), (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cnt += 1
        else:
            # 얼굴이 없거나 1 이상인 경우
            if len(faces) == 0:
                msg = "no face."
            elif len(faces) > 1:
                msg = "too many face."
            cv2.putText(frame, msg, (10, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255))
        cv2.imshow('face record', frame)
        if cv2.waitKey(1) == 27 or cnt == target_cnt:
            break
cap.release()
cv2.destroyAllWindows()
print("Collecting Samples Completed")

Insert User Name(Only Alphabet): sk
Insert User Id(Non-Duplicate number): 0


## LBP 얼굴 인식기 훈련

In [31]:
import cv2
import numpy as np
import os, glob

In [34]:
base_dir = './faces'
train_data, train_labels = [], []
dirs = [d for d in glob.glob(base_dir+"/*") if os.path.isdir(d)]
print('Collecting train data set:')
for dir in dirs:
    id = dir.split('_')[1]
    files = glob.glob(dir+'/*.jpg')
    print('\t path:%s, %dfiles'%(dir, len(files)))
    for file in files:
        img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
        train_data.append(np.asarray(img, dtype = np.uint8))
        train_labels.append(int(id))
        
train_data = np.asarray(train_data)
train_labels = np.int32(train_labels)

print('Strating LBP Model training...')
model = cv2.face.LBPHFaceRecognizer_create()
model.train(train_data, train_labels)
model.write('./faces/all_face.xml')
print("Model trained successfully")
                      

	 path:./faces/sk_0, 400files
Strating LBP Model training...
Model trained successfully


## 훈련된 LBP 얼굴 인식기로 얼굴 인식

In [35]:
import cv2
import numpy as np
import os, glob

In [36]:
base_dir = './faces'
min_accuracy = 85

face_classifier = cv2.CascadeClassifier('./data/haarcascade_frontalface_default.xml')
model = cv2.face.LBPHFaceRecognizer_create()
model.read(os.path.join(base_dir, 'all_face.xml'))

dirs = [d for d in glob.glob(base_dir+'/*') if os.path.isdir(d)]
names = dict([])
for dir in dirs:
    dir = os.path.basename(dir)
    name, id = dir.split('_')
    names[int(id)] = name
    
cap = cv2.VideoCapture(-1)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print('no frame')
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        label, confidence = model.predict(face)
        if confidence < 400:
            accuracy = int(100*(1-confidence/400))
            if accuracy >= min_accuracy:
                msg = '%s(%.0f%%)'%(names[label], accuracy)
            else:
                msg = 'Unknown'
            
        txt, base = cv2.getTextSize(msg, cv2.FONT_HERSHEY_PLAIN, 1, 3)
        cv2.rectangle(frame, (x, y-base-txt[1]), (x+txt[0], y+txt[1]), (0, 255, 255), -1)
        cv2.putText(frame, msg, (x, y), cv2.FONT_HERSHEY_PLAIN, 1, (200, 200, 200), 2, cv2.LINE_AA)
    cv2.imshow('Face Recognition', frame)
    if cv2.waitKey(1) == 27:
        break
cap.release()
cv2.destroyAllWindows()